# Romania Kendo Stats  
#### 25 years of Kendo History in Romania, visualized

# Data cleaning workbook

Created by **[Dénes Csala](//csaladen.es)** | 2019 | MIT License 
  
For any improvement suggestions and spotted processing mistakes drop me a message on [Facebook](//facebook.com/csaladenes).  
If you would like to have your country/club data visualized in a similar manner, or any other data visualization and analytics consultancy inquiries contact me at [mail@csaladen.es](mailto:mail@csaladen.es)

This workbook guides you through the data cleaning stage for the **Romania Kendo Stats** visualization. This is a multi-stage process, you will need access to the raw data (liaise with Secretary or other member in charge of data the [Romanian Kendo Association](https://www.kendo-romania.ro/)), [Python](https://www.python.org/) and [Excel](https://products.office.com/en-us/excel)  installed. Any Python packages will also be installed on the way, but we recommend using the [Anaconda](https://www.anaconda.com/download/) distribution of _Python 3_. If you would like to edit the visualization part, then you will need [PowerBI Desktop](https://powerbi.microsoft.com/en-us/desktop/).

The general structure of the repository is the following:  
 - `/data`
   - `/raw`: this where you place the downloaded data from the official data source, sorted by years and competitions, only keep those that have relevant data for matches only
   - `/ocr`: this is where the data gets saved after an [OCR](https://en.wikipedia.org/wiki/Optical_character_recognition) has been performed - this is necessary for some older files in image format 
   - `/manual`: this is where manually extracted matches from old image files get placed - they should follow the `2018 CN` format, i.e. all matches in one sheet
   - `/export`: this is where we save the dataformatted for loading into the viz
   - `/clean`: this is where all the processed, cleaned data ends up - they should follow the `2018 CN` format, i.e. all matches in one sheet
 - `/scripts`: this is the main code repository for all data processing scripts
 - `/viz`: this is where the visualization files get saved - they are created using PowerBI and load data from `/data/clean`

## 1. Load and clean members

This section reads and clean the _RKA_ members list. Save as baseline. 

In [1]:
import pandas as pd, numpy as np, json
import members_loader, matches_loader, clubs_loader, point_utils

First, download members data (`Evidenta membrilor.xlsx`) from the official data source, and create a macro-enabled Excel file from the Google Sheet. Then write a simple macro to extract the cell comments from the _Club_ column in order to get info about club _Transfers_. Follow the instructions [here](https://www.extendoffice.com/documents/excel/765-excel-convert-comments-to-cells.html). Save the new file as `Evidenta membrilor.xlsm` in the `/data/manual` folder. Use the `members_loader` module to process this file.

In [2]:
members=members_loader.get_members('../data/manual/Evidenta membrilor.xlsm')

Members are loaded but a bit messy.

In [3]:
members.head(2)

,231,Nr. EKF,Club,Unnamed: 3,Numele,Prenumele,Gen,Data naşterii,1 kyu,practică,1 dan,2 dan,3 dan,4 dan,5 dan,6 dan,7 dan,8 dan
0,Activ,RO.00205,TAI,NaN,Abrudan,Dorin-Ștefan,M,1991-12-27,2015-08-08 00:00:00,NaN,2015-12-19,NaT,NaT,NaT,NaT,NaT,NaN,NaN
1,Inactiv,RO.00083,ICH,Transfer: CRK => ICH - 2009,Ah-hu,Weizhi Stéphen,M,1980-06-12,NaN,NaN,2009-08-01,2010-09-26,NaT,NaT,NaT,NaT,NaN,NaN


In [4]:
members_clean=members_loader.cleaner(members).reset_index(drop=False)

In [5]:
members_clean.to_csv('../data/clean/members.csv')

## 2. Load and clean matches

Matches are loaded from excel sheets in the `/data` folder, organized by year and competition. We are always looking for match list data,the cleaner the better, the more concentrated the better. While this is not possible all the time, we have several demo import routines. These are stored in the `matches_loader.py` function library. While not all matches have textual data available, these will need to be processed through OCR first. Raw excel data that can be processed right away can be found in the `/data/raw` folder, while the processed ones in `/data/ocr`. We use a separate workbook, `ocr.ipynb` to walk you through the OCR process.

In [6]:
matches={i:{} for i in range(1993,2019)}
competitions={
    2018:['CR','CN','SL'],
    2017:['CR','CN','SL'],
    2016:['CR','CN','SL'],
    2015:['CR','CN','SL'],
    2014:['CR','CN','SL'],
    2013:['CR','CN','SL'],
    2012:['CR','CN'],
    2011:['CR','CN'],
    2010:['CR','CN'],
    2009:['CR','CN'],
    1998:['CR'],
    1997:['CR'],
    1993:['CR']
}

### 2.1. Load matches

In [7]:
for year in competitions:
    for competition in competitions[year]:
        matches[year][competition]=matches_loader.get_matches(year,competition)

[ 0  3  4  7  8 11 12 15 16 19 20 23 24 27 28 31 32 35 36 39 40 43 44 47
 48 51 52 55 56 59 60 63]
[ 1  6  9 14 17 22 25 30 33 38 41 46 49 54 57 62]
[ 3 12 19 28 35 44 51 60]
[ 7 24 39 56]
[15 48]
[ 0  3  4  7  8 11 12 15 16 19 20 23 24 27 28 31 32 35 36 39 40 43 44 47
 48 51 52 55 56 59 60 63]
[ 1  6  9 14 17 22 25 30 33 38 41 46 49 54 57 62]
[ 3 12 19 28 35 44 51 60]
[ 7 24 39 56]
[15 48]
[ 0  3  4  7  8 11 12 15 16 19 20 23 24 27 28 31 32 35 36 39 40 43 44 47
 48 51 52 55 56 59 60 63]
[ 1  6  9 14 17 22 25 30 33 38 41 46 49 54 57 62]
[ 3 12 19 28 35 44 51 60]
[ 7 24 39 56]
[15 48]


In [8]:
year, competition

(1993, 'CR')

In [9]:
matches[2009]['CR'][8]

{'match_type': '../data/raw/2009/CR/2009.04.04 - CR - Budeasa - print.xlsx#IM$pool%nan',
 'aka': {'name': 'Antal Róbert', 'point1': nan},
 'shiro': {'name': 'Niciev Cristian', 'point1': 'K'},
 'outcome': 1}

### 2.2. Standardize names

Names in `name_exceptions` get replaced with their right hand side values _before_ processing.

In [10]:
name_exceptions={'Atanasovski':'Atanasovski A. (MAC)',
                 'Dobrovicescu (SON)':'Dobrovicescu T. (SON)',
                 'Ianăș':'Ianăș F.',
                 'Crăciun (Tamang) Sujata':'Crăciun S.',
                 'Dinu (Ioniță) Claudia-Andreea':'Dinu A.',
                 'Arabadjiyski': 'Arabadjiyski A.',
                 'Mandia':'Mandia F.',
                 'Stanev':'Stanev A.',
                 'Mochalov':'Mochalov O.',
                 'Sozzi':'Sozzi A.',
                 'Crăciunel':'Crăciunel I.',
                 'Craciunel':'Crăciunel I.',
                 'Sagaev':'Sagaev L.',
                 'Buzás':'Búzás C.',
                 'Csala':'Csala T.',
                 'Dimitrov':'Dimitrov M.',
                 'Józsa':'Józsa L.',
                 'Creangă':'Creangă A.',
                 'Duțescu':'Duțescu M.',                 
                 'Furtună':'Furtună G.',
                 'Gârbea':'Gârbea I.',
                 'Stupu':'Stupu I.',
                 'Mahika-Voiconi':'Mahika-Voiconi S.',
                 'Mahika':'Mahika-Voiconi S.',
                 'Stanciu':'Stanciu F.',
                 'Vrânceanu':'Vrânceanu R.',
                 'Wolfs':'Wolfs J.',
                 'Ducarme':'Ducarme A.',
                 'Sbârcea':'Sbârcea B.',
                 'Mocian':'Mocian A.',
                 'Hatvani':'Hatvani L.',
                 'Dusan':'Dusan N.',
                 'Borota':'Borota V.',
                 'Tsushima':'Tsushima K.',
                 'Tráser':'Tráser T.',
                 'Colțea':'Colțea A.',
                 'Brîcov':'Brîcov A.',
                 'Yamamoto':'Yamamoto M.',
                 'Crăciun':'Crăciun D.'}

Names in `name_equals` get replaced with their right hand side values _after_ processing.

In [11]:
name_equals={'Chirea M.':'Chirea A.',
            'Ghinet C.':'Ghineț C.',
            'Anghelescu A.':'Anghelescu M.',
            'Domnița M.':'Domniță M.',
            'Bejgu N.':'Beygu N.',
            'Canceu A.':'Canceu Ad.',
            'Dinu C.':'Dinu A.',
            'Grapa D.':'Grapă D.',
            'Cristea C.':'Cristea Că.',
            'Cismas O.':'Cismaș O.',
            'Garbea I.':'Gârbea I.',
            'Ah-hu W.':'Ah-hu S.',
            'Horvát M.':'Horváth M.',
            'Ionita A.':'Ioniță A.',
            'Medvedschi I.':'Medvețchi I.',
            'Mahika S.':'Mahika-Voiconi S.',
            'Mate L.':'Máté L.',
            'Hentea L.':'Hentea A.',
            'Stupu I.':'Stupu A.',
            'Ah-Hu S.':'Ah-hu S.',
            'Alexa I.':'Alexa A.',
            'Albert V.':'Albert J.',
            'Angelescu M.':'Angelescu M.',
            'Apostu D.':'Apostu T.',
            'Brâcov A.':'Brîcov A.',
            'Zaporojan R.':'Zaporojan O.',
            'Vasile C.':'Vasile I.',
            'Tudor-Duicu C.':'Tudor D.',
            'Țarălungă D.':'Țarălungă A.',
            'Sandu M.':'Sandu Mar.',
            'Radulescu A.':'Rădulescu A.',
            'Péter C.':'Péter Cso.',
            'Movatz E.':'Movatz V.',
            'Molinger B.':'Molinger P.',
            'Mitelea C.':'Mițelea C.',
            'Macavei I.':'Macaveiu I.',
            'Luca T.':'Luca Tr.',
            'Leca L.':'Leca F.',
            'Gutu E.':'Guțu E.',
            'Mehelean L.':'Mahalean L.',
            'Catoriu D.':'Cantoriu D.',
            'Călina A.':'Călina C.',
            'Buzás C.':'Búzás C.',
            'Korenshi E.':'Korenschi E.',
            'Pleșa R.':'Pleșea R.',
            'Galos A.':'Galoș A.',
            'Győrfi G.':'Györfi G.',
            'Győrfi S.':'Györfi S.',
            'Ghineț G.':'Ghineț C.',
            'Hostina E.':'Hoștină E.', 
            'Hostină E.':'Hoștină E.', 
            'Ianăs F.':'Ianăș F.',
            'Ianas F.':'Ianăș F.',
            'Taralunga D.':'Țarălungă D.',
            'Lacatus M.':'Lăcătuș M.',
            'Máthé L.':'Máté L.',
            'Burinaru A.':'Burinaru Al.',
            'Nastase M.':'Năstase E.',
            'Oprisan A.':'Oprișan A.',
            'Pârlea A.':'Pîrlea A.',
            'Sabau D.':'Sabău D.',
            'Spriu C.':'Spiru C.',
            'Bíró S.':'Biró S.',
            'Stănculascu C.':'Stănculescu C.',
            'Vrânceanu M.': 'Vrânceanu L.',
            'Wasicek V.':'Wasicheck W.',
            'Wasicsec W.':'Wasicheck W.',
            'Wasicsek W.':'Wasicheck W.',
            'Zolfoghari A.':'Zolfaghari A.'}

Names in `name_doubles` handle situation where the default name abbreviation might lead to duplicates.

In [12]:
name_doubles={
        'Cristea Cristina':'Cristea Cr.', 
        'Cristea Călin-Ștefan':'Cristea Că.',
        'Sandu Marius-Cristian':'Sandu Mar.', 
        'Sandu Matei-Serban':'Sandu Mat.',
        'Georgescu Andrei':'Georgescu An.', 
        'Georgescu Alexandra':'Georgescu Al.',
        'Péter Csongor':'Péter Cso.', 
        'Péter Csanád':'Péter Csa.',
        'Luca Mihnea':'Luca Mihn.', 
        'Luca Mihai-Cătălin':'Luca Miha.',
        'Luca':'Luca Miha.',
        'Luca M':'Luca Miha.',
        'Luca M.':'Luca Miha.',
        'Luca Mihai':'Luca Miha.',
        'Luca Traian-Dan':'Luca Tr.', 
        'Luca Tudor':'Luca Tu.',
        'Canceu Anamaria':'Canceu An.', 
        'Canceu Adriana-Maria':'Canceu Ad.',
        'Cioată Daniel-Mihai':'Cioată M.', 
        'Cioată Dragoș':'Cioată D.',
        'Burinaru Alexandra':'Burinaru Al.', 
        'Burinaru Andreea':'Burinaru An.',
        'Dudaș Francisc Andrei':'Dudaș F.', 
        'Dudaș Francisc':'Dudaș F.'}

Normalize Romanian characters, define name cleaner function to get _Name IDs_. Name ID are unique competitor names in the form of: _Surname, First letter of Name_. If the First Letter of Name leads to a non-unique ID, the second letter is taken, and so forth, until a unique ID is found. It gets contructed as follows:
 1. If name in doubles return the solution directly
 2. Normalize characters
 3. If name is in exceptions, clean
 4. Replace any double spaces, then split at _(_ (to split away club, if embedded in the name)
 5. Split into Surname and Name, store in `rnames`
 6. Store _Surname N._ in `sname`
 7. If `sname` is in equals, clean
 8. Retrun `sname`

In [13]:
letter_norm={'ţ':'ț','ş':'ș','Ş':'Ș'}
def name_cleaner(name):
    name=str(name)
    if name in name_doubles:
        return name_doubles[name]
    else:
        for letter in letter_norm:
            name=name.replace(letter,letter_norm[letter])
        if name in name_exceptions:
            name=name_exceptions[name]
        nc=name.replace('  ',' ').split('(')    
            
        rname=nc[0].strip()
        rnames=rname.split(' ')
        sname=rnames[0]+' '+rnames[1][0]+'.'
        if sname in name_equals:
            sname=name_equals[sname]
        if sname in name_doubles:
            print(name,sname)
    return sname

Names equalling any string in `redflags_names` get thrown out of the final dataset.  
Names containing any string in `redflags_names2` get thrown out of the final dataset.

In [14]:
redflags_names=['-','—','—',np.nan,'. ()','— ','- -.','- -. (-)','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','R','S',
                'Kashi','Sankon','București','Victorii:','Sakura','Taiken','Ikada','Sonkei','CRK','Museido',
                'Ichimon','Bushi Tokukai 1','Competitori – Shiai-sha','Echipa - roşu','Numele şi prenumele',
                'Victorii:','Victorii: 0','Victorii: 1','Victorii: 2','Victorii: 3','Victorii: 4',
                'Victorii: 5','?','Kyobukan','2/5','2/6','3/8','Finala','Kyobukan (0/0/0)','―',
                '(clasament final după meci de baraj)','CRK (Bucuresti)','Kaybukan','Isshin (Cluj)',
                'Ikada (Bucureşti)','Kyobukan (Braşov)','Puncte:','KASHI','Budoshin','Isshin',
                '— (—)','4. B.','4. Baraj: Stupu M - Hostina','4. Baraj: Moise KM - Korenschi M',
               'Bushi Tokukai (2/8/17)','CRK 2 (1/6/14)', 'CRK 2','CRK 1','Loc I.:','Loc',
               'Bushi Tokukai 2 (M Ciuc)','Echipa suport']
redflags_names2=['Bushi Tokukai','Eliminatoriu','finala','Finala','Fianala','Ikada','Ichimon','Pool',
                'Locul ','Lotul ','Loc ','Grupa ','Isshin','Meciul ','Victorii:','L1','1','2','3','4','5','6','7','8','9','0']

Check is name is not in redflags. Ignore these entries.

In [15]:
def name_ok(name):
    name=str(name)
    if name=='nan': return False
    if name not in redflags_names:
        if np.array([i not in name for i in redflags_names2]).all():
            return True
    return False

Process all names for standardization. Create 3 variables:  
1. `all_players`: forward relationship: unclean name -> cleaned name
2. `all_players_r`: reverse relationship
3. `all_players_unsorted`: unique set of all names processed 

Process both competitor and shinpan names.

In [16]:
all_players={}
all_players_r={}
all_players_unsorted=set()
for year in matches:
    for competition in matches[year]:
        for match in matches[year][competition]:
            for color in ['aka','shiro']:
                name=match[color]['name']
                all_players_unsorted.add(name)
                if name_ok(name):
                    name=name_cleaner(name)
                    rname=match[color]['name']
                    if rname not in all_players_r:all_players_r[rname]=name
                    if name not in all_players: all_players[name]={}
                    if year not in all_players[name]:all_players[name][year]={'names':set()}
                    all_players[name][year]['names'].add(rname)
            if 'shinpan' in match:
                for color in ['fukushin1','shushin','fukushin2']:
                    aka=match['aka']['name']
                    shiro=match['shiro']['name']
                    if (name_ok(aka)) and\
                       (name_ok(shiro)) and\
                       (name_cleaner(aka) in all_players) and\
                       (name_cleaner(shiro) in all_players):
                        rname=match['shinpan'][color]
                        all_players_unsorted.add(rname)
                        if name_ok(rname):
                            name=name_cleaner(rname)
                            if rname not in all_players_r:all_players_r[rname]=name
                            if name not in all_players: all_players[name]={}
                            if year not in all_players[name]:all_players[name][year]={'names':set()}
                            all_players[name][year]['names'].add(rname)

Link procesed to names in `members`. The `name_linker` dictionary contains Name IDs (short names) as keys and sets of long names as values. Ideally, this set should contain only one element, so that the mapping in unique.

In [17]:
name_linker={}
for i in members_clean.index:
    name=members_clean.loc[i]['name']
    try:
        cname=name_cleaner(name)
    except:
        print(name)
    if cname not in name_linker:name_linker[cname]=set()
    name_linker[cname].add(name)

Do the opposite mapping in `names_abbr`: long->short. Create exceptions for duplicate names.

In [18]:
names_abbr={}
for name in name_linker:
    if len(name_linker[name])>1:
        #only for dev to create exceptions for duplicate person names.
        print(name,name_linker[name])
    for i in name_linker[name]:
        names_abbr[i]=name

Dudaș F. {'Dudaș Francisc', 'Dudaș Francisc Andrei'}


Save club mappings by short name, by year.

In [19]:
names_abbr_list=[]
name_abbr2long={}
name_abbr2club={}
for i in members_clean.index:
    name=members_clean.loc[i]['name']
    club=members_clean.loc[i]['club']
    year=members_clean.loc[i]['year']
    names_abbr_list.append(names_abbr[name])
    name_abbr2long[names_abbr[name]]=name
    if names_abbr[name] not in name_abbr2club:name_abbr2club[names_abbr[name]]={}
    name_abbr2club[names_abbr[name]][year]=club

Add short names to `members_clean`.

In [20]:
members_clean['name_abbr']=names_abbr_list

Some names appear in the short form, we need to add them manually to the long list. We parse through all forms in which the name appears, and choose the longest. We call this the inferred name.

In [21]:
for name in all_players:
    if name not in name_abbr2long:
        #infer using longest available name
        names={len(j):j  for i in all_players[name] for j in all_players[name][i]['names']}
        if len(names)>0:
            inferred_name=names[max(names.keys())]
            if '(' in inferred_name:
                inferred_name=inferred_name[:inferred_name.find('(')-1]
            name_abbr2long[name]=inferred_name

### 2.3. Infer clubs

Infer clubs from name if club is part of name in the competition. Club names in `redflags_clubs` get ignored. Clubs in `club_equals` get replaced _after_ processing. The convention is to have 3 letter all-caps club names for Romanian clubs, 3 letter club names followed by a / and a two letter country code for foreign clubs.

In [22]:
redflags_clubs=['','N/A','RO1','RO2']
club_equals={'MLD':'MOL/Md',
             'MOL':'MOL/Md',
             'IKD':'IKA',
             'HUN':'HUN/Hu',
             'BUL':'BUL/Bg',
             'TUR':'TUR/Tr',
             'MAC':'MAC/Mc',
             'MNE':'MNE/Mn',
             'SRB':'SRB/Sr',
             'ITA':'ITA/It',
             'ISS':'ISH',
             'Musso, Bg':'MUS/Bg',
             'Makoto, Sr':'MAK/Sr',
             'Szeged, Hu':'SZE/Hu'}

Attach clubs to all_players who have it in their competition name data.

In [23]:
for name in all_players:
    #if we dont already know the club for this year from the members register
    if name not in name_abbr2club:    
        for year in all_players[name]:
            for name_form in all_players[name][year]['names']:
                if '(' in name_form:
                    club=name_form.split('(')[1].strip()[:-1]
                    if club not in redflags_clubs:
                        if name not in name_abbr2club:name_abbr2club[name]={}
                        name_abbr2club[name][year]=club
    else:
        for year in all_players[name]:
            #ese if no club info for particular year
            if year not in name_abbr2club[name]:
                for name_form in all_players[name][year]['names']:
                    if '(' in name_form:
                        club=name_form.split('(')[1].strip()[:-1]
                        if club not in redflags_clubs:
                            name_abbr2club[name][year]=club

Normalize club names and long names.

In [24]:
for name in name_abbr2club:
    for year in name_abbr2club[name]:
        if name_abbr2club[name][year] in club_equals: 
            name_abbr2club[name][year]=club_equals[name_abbr2club[name][year]]
for name in name_abbr2long:
    name_abbr2long[name]=name_abbr2long[name].replace('  ',' ').strip()

If club still not found, fill the gaps between years. Forward fill first, then backward fill, if necessary.

In [25]:
manual_club_needed=set()
for name in all_players:
    if name in name_abbr2club:
        years=np.sort(list(all_players[name].keys()))
        minyear1=min(years)
        maxyear1=max(years)
        minyear2=min(name_abbr2club[name].keys())
        maxyear2=min(name_abbr2club[name].keys())
                    
        if len(years)>1:
            for year in range(min(minyear1,minyear2),max(maxyear1,maxyear2)+1):
                if year not in name_abbr2club[name]:
                    #get club from previous year
                    for y in range(years[0],year):
                        if y in name_abbr2club[name]:
                            name_abbr2club[name][year]=str(name_abbr2club[name][y])
                            break
                if year not in name_abbr2club[name]:
                    #if still not found, get club from next year
                    for y in np.arange(years[-1],year,-1):
                        if y in name_abbr2club[name]:
                            name_abbr2club[name][year]=str(name_abbr2club[name][y])
                            break
                if year not in name_abbr2club[name]:
                    #if still not found, get first known year
                    if year<minyear2:
                        name_abbr2club[name][year]=str(name_abbr2club[name][minyear2])
                    else:
                        name_abbr2club[name][year]=str(name_abbr2club[name][maxyear2])
            #now interpolate years with missing data
            #for year in name_abbr2club[name]:
    else:
        manual_club_needed.add(name)

We have extracted what was possible from the data. Now we do a save of short name to long name and cllub mappings (by year). We then edit this file manually, if necessary.

### 2.4. Manual club and long name overrides

In [26]:
manual_name_needed=set()
#check if we dont have first name information, then flag for manual additions
for name in name_abbr2long:
    names=name_abbr2long[name].split(' ')
    if len(names)<2:
        manual_name_needed.add(name)
    elif len(names[1])<3:
        manual_name_needed.add(name) 

In [27]:
manual_data_override=pd.read_excel('../data/manual/members_manual.xlsx').set_index('name')

In [28]:
manual_data_needed=[]
for i in manual_name_needed.union(manual_club_needed):
    if i not in list(manual_data_override.index):
        dummy={'name':i,'long_name':'','club':''}
        if i in name_abbr2club:
            dummy['club']=name_abbr2club[name][max(list(name_abbr2club[name].keys()))]
        if i in manual_club_needed:
            if i in name_abbr2long:
                dummy['long_name']=name_abbr2long[i]
        manual_data_needed.append(dummy)

In [29]:
df=pd.DataFrame(manual_data_needed).set_index('name')
df=pd.concat([manual_data_override,df]).drop_duplicates().sort_index()

In [30]:
df.to_excel('../data/manual/members_manual.xlsx')

Extend with manual data

In [31]:
for i in df['long_name'].replace('',np.nan).dropna().index:
    name_abbr2long[i]=df.loc[i]['long_name']
    all_players_r[name_abbr2long[i]]=i

In [32]:
manual_club_needed=set()
for name in all_players:
    years=np.sort(list(all_players[name].keys()))
    minyear=min(years)
    maxyear=max(years)
    for year in range(minyear,maxyear+1):
        if name not in name_abbr2club:name_abbr2club[name]={}
        if year not in name_abbr2club[name]:
            if name in df['club'].replace('',np.nan).dropna().index:
                name_abbr2club[name][year]=df.loc[name]['club']
            else:
                name_abbr2club[name][year]='XXX'

## 3. Update members  
Extend members data with data mined from matches

Create club cleaner function. Split club from country by /.

In [33]:
def club_cleaner(club,country='RO'):
    if club in club_equals:
        club=club_equals[club]
    if str(club)=='nan':
        club='XXX'
    if '/' in club:
        return club.split('/')[0],club.split('/')[1].upper()
    else:
        return club,country

Create club long name translator.

In [34]:
pretty_clubs={'ARA':'Arashi', 'ACKIJ':'Official', 'BDS':'Budoshin', 'BSD':'Bushido', 'BTK':'Bushi Tokukai', 'BG':'Bulgaria',
              'CDO':'Coroana de Oțel', 'CRK':'Clubul Român de Kendo', 'HAR':'Hargita', 
              'ICH':'Ichimon', 'IKA':'Ikada','ISH':'Ishhin', 'IT':'Italy','HU':'Hungary',
              'KAS':'Kashi', 'KNS':'Kenshin', 'KYO':'Kyobukan', 'MC':'Macedonia',
              'SR':'Serbia', 'MN':'Montenegro', 'MD':'Moldova', 'MUS':'Museido', 
               'RON':'Ronin-do', 'SAK':'Sakura', 'SAM':'Sam-sho','SAN':'Sankon', 'SBK':'Sobukan',
               'SON':'Sonkei', 'SR':'Serbia','BE':'Belgium', 'TAI':'Taiken', 'TR':'Turkey', 'XXX':'Unknown',
               'YUK':'Yu-kai','KAY':'Kaybukan'}
def pretty_club(club, country):
    if country!='RO':
        return pretty_clubs[country]
    else: return pretty_clubs[club]

Extend members with unregistered members. Either inactive, or from abroad.

In [35]:
unregistered_members=[]
for name in all_players:
    if name not in set(members_clean['name_abbr'].values):
        years=np.sort(list(name_abbr2club[name].keys()))
        for year in range(min(years),max(years)+1):
            if year in all_players[name]:
                iyear=year
            else:
                iyear=max(years)
            club,country=club_cleaner(name_abbr2club[name][year])
            if country=='RO':
                activ='Inactiv'
                dan=''#dan=0
            else:
                activ='AS'
                dan=''
            unregistered_members.append({'name':name_abbr2long[name],'name_abbr':name,
                    'club':club,'active':activ,'year':year,'dan':dan,'country':country,
                    'pretty_club':pretty_club(club,country)})

In [36]:
members_clean['country']='RO'

In [37]:
members_updated=pd.concat([members_clean,pd.DataFrame(unregistered_members)]).reset_index(drop=True)

C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [38]:
clubs=[]
pclubs=[]
countries=[]
for i in members_updated.index:
    club=members_updated.loc[i]['club']
    country=members_updated.loc[i]['country']
    cc=club_cleaner(club,country)
    clubs.append(cc[0])
    pclubs.append(pretty_club(cc[0],cc[1]))
    countries.append(cc[1])

In [39]:
members_updated['club']=clubs
members_updated['pretty_club']=pclubs
members_updated['country']=countries

Extend 0 dan down to starting year.

In [40]:
members_mu_dan_extensions=[]
members_by_name=members_updated.set_index(['name_abbr'])
for year in matches:
    members_by_year=members_updated.set_index(['year']).loc[year]
    for competition in matches[year]:
        print(year,competition)
        for k in matches[year][competition]:
            aka=k['aka']['name']
            shiro=k['shiro']['name']
            if (name_ok(aka)) and\
               (name_ok(shiro)) and\
               (name_cleaner(aka) in all_players) and\
               (name_cleaner(shiro) in all_players):
                for a in ['aka','shiro']:
                    for h in k[a]:
                        if h=='name':
                            name=k[a][h]
                            rname=all_players_r[name]
                            if rname in list(members_by_name.index):
                                if rname not in members_by_year['name_abbr'].values:
                                    dummy=members_by_name.loc[[rname]]
                                    minyear=min(dummy['year'])
                                    maxyear=max(dummy['year'])
                                    if year>maxyear:
                                        dummy=dummy[dummy['year']==maxyear]
                                        yeardiff=min(dummy['year'])-year
                                    else:
                                        dummy=dummy[dummy['year']==minyear]
                                        yeardiff=year-max(dummy['year'])
                                    dummy=dummy.reset_index()
                                    dummy['year']=year
                                    dummy['age']=dummy['age']+yeardiff
                                    members_mu_dan_extensions.append(dummy)
                            #if only appears in competition in one year, then not in members table
                            else:
                                print(rname,year)
                                #fix in unregistered_members

1993 CR
1997 CR
1998 CR
2009 CR
2009 CN
2010 CR
2010 CN
2011 CR
2011 CN
2012 CR
2012 CN
2013 CR
2013 CN
2013 SL
2014 CR
2014 CN
2014 SL
2015 CR
2015 CN
2015 SL
2016 CR
2016 CN
2016 SL
2017 CR
2017 CN
2017 SL
2018 CR
2018 CN
2018 SL


Update members

In [41]:
members_mu_dan_extensions=pd.concat(members_mu_dan_extensions)
members_updated=pd.concat([members_updated,members_mu_dan_extensions])

C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Fix unknwown genders

In [42]:
manual_mf_data_override=pd.read_excel('../data/manual/members_mf_manual.xlsx')

In [43]:
manual_mf_data_needed=members_updated[(members_updated['gen']!='M')&(members_updated['gen']!='F')][['name_abbr','name']]\
        .drop_duplicates()

In [44]:
df=manual_mf_data_needed.merge(manual_mf_data_override[['name_abbr','gen']],'outer').drop_duplicates()
df.to_excel('../data/manual/members_mf_manual.xlsx')

Update members with manual gender data.

In [45]:
members_updated=members_updated.reset_index(drop=True).drop_duplicates()

In [46]:
gens=[]
for i in members_updated.index:
    name=members_updated.loc[i]['name_abbr']
    if name in list(df.index):
        gens.append(df.loc[name])
    else:
        gens.append(members_updated.loc[i]['gen'])

In [47]:
members_updated['gen']=gens

Save to `/data/export`.

In [48]:
members_updated.to_csv('../data/export/members.csv')

## 4. Update matches  
Update and save cleaned match data

In [49]:
redflags_points=['Puncte']

Check if outcome was encho.

In [50]:
def outcome_cleaner(outcome):
    if outcome=='E': return True
    else: return False

Check who won, by how many points.

In [51]:
def outcome_from_points(aka,shiro):
    if aka==shiro: return 'X',0
    elif aka>shiro: return 'A',str(aka-shiro)
    else: return 'S',str(shiro-aka)

Create master match variable. Connect this to the viz.

In [52]:
master_matches=[]
for year in matches:
    members_by_year=members_updated.set_index(['year']).loc[year].drop_duplicates()
    for competition in matches[year]:
        print(year,competition)
        for k in matches[year][competition]:
            good=True
            match={'year':year,'competition':competition}
            match['match_category'],match['match_teams'],match['match_phase']=point_utils.match_cleaner(year,k['match_type'])
            if 'shinpan' in k:
                for color in ['fukushin1','shushin','fukushin2']:
                    if color in k['shinpan']:
                        if k['shinpan'][color] in all_players_r:
                            #normalize shinpan names
                            match[color]=name_abbr2long[all_players_r[k['shinpan'][color]]]
            aka=k['aka']['name']
            shiro=k['shiro']['name']
            if (name_ok(aka)) and\
               (name_ok(shiro)) and\
               (name_cleaner(aka) in all_players) and\
               (name_cleaner(shiro) in all_players):
                for a in ['aka','shiro']:
                    points=''
                    for h in k[a]:
                        if h=='name':
                            name=k[a][h]
                            #normalize competitor names
                            rname=all_players_r[name]
                            df=members_by_year[members_by_year['name_abbr']==rname]
                            match[a+' name']=name_abbr2long[rname]
                        else:
                            point=k[a][h]
                            if str(point)=='nan': point=''
                            points=points+point
                    for redflag in redflags_points:
                        if redflag in points:
                            good=False
                    if good:
                        match[a+' point1'],match[a+' point2'],match[a+' points'],\
                                match[a+' hansoku'],match['encho']=point_utils.points_cleaner(points)
            else:
                good=False                
            if good:
                if 'outcome' in k:
                    match['encho']=outcome_cleaner(k['outcome'])
                else: 
                    match['encho']=False
                match['winner'],match['difference']=outcome_from_points(match['aka points'],match['shiro points'])

                master_matches.append(match)

1993 CR
1997 CR
1998 CR
2009 CR
2009 CN
2010 CR
2010 CN
2011 CR
2011 CN
2012 CR
2012 CN
2013 CR
2013 CN
2013 SL
2014 CR
2014 CN
2014 SL
2015 CR
2015 CN
2015 SL
2016 CR
2016 CN
2016 SL
2017 CR
2017 CN
2017 SL
2018 CR
2018 CN
2018 SL


Clean up matches for pretty display

In [53]:
data=pd.DataFrame(master_matches).reset_index(drop=True)

In [54]:
data['aka hansoku']=data['aka hansoku'].replace(0,'').replace(1,'Δ')
data['shiro hansoku']=data['shiro hansoku'].replace(0,'').replace(1,'Δ')

In [55]:
data.to_csv('../data/export/matches.csv')

Group by player

In [56]:
aka=data[[i for i in data.columns if 'shiro ' not in i]]
aka.columns=[i.replace('aka ','') for i in aka.columns]
aka['color']='aka'
aka['opponent']=data['shiro name']
aka=aka.reset_index()

C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [57]:
shiro=data[[i for i in data.columns if 'aka ' not in i]]
shiro.columns=[i.replace('shiro ','') for i in shiro.columns]
shiro['color']='shiro'
shiro['opponent']=data['aka name']
shiro=shiro.reset_index()

C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [58]:
extended_matches=pd.concat([aka,shiro],axis=0).reset_index(drop=True)

C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [62]:
extended_matches.to_csv('../data/export/extended_matches.csv')

Save data by points

In [63]:
p1=extended_matches[[i for i in extended_matches.columns if i!='point2']]
p2=extended_matches[[i for i in extended_matches.columns if i!='point1']]
p1.rename(columns={'point1':'point'}, inplace=True)
p2.rename(columns={'point2':'point'}, inplace=True)

C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [64]:
extended_points=pd.concat([p1,p2],axis=0).reset_index(drop=True)

In [65]:
extended_points.to_csv('../data/export/extended_points.csv')

Save shinpan data - most comprehensive

In [66]:
shu=extended_points[[i for i in extended_points.columns if 'fukushin' not in i]]
shu.columns=[i.replace('shushin','shinpan') for i in shu.columns]
fk1=extended_points[[i for i in extended_points.columns if 'shushin' not in i and 'fukushin2' not in i]]
fk1.columns=[i.replace('fukushin1','shinpan') for i in fk1.columns]
fk2=extended_points[[i for i in extended_points.columns if 'shushin' not in i and 'fukushin1' not in i]]
fk2.columns=[i.replace('fukushin2','shinpan') for i in fk2.columns]
extended_shinpan=pd.concat([shu,fk1,fk2],axis=0).reset_index(drop=True)

C:\Users\csala\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [67]:
extended_shinpan.to_csv('../data/export/extended_shinpan.csv')